# The Word Ladder Game
Author: Jake Kougan

Class project from Computer Science 128 at Illinois Wesleyan Universty

In [ ]:
#Import Statements
import string
import networkx as nx
from matplotlib import pyplot as plt
from collections import deque
from networkx.classes.graph import Graph
import random



# Helper Functions

These functions deal specifically with the words which are a crucial part of the word ladder game

In [5]:


def x_length_words(word_lst, word_len):
  '''Takes a list and number and returns a new list with containing only the words from the original list that have the same length

  Parameters:
    word_lst (list): the original list of words
    word_len (int): the desire length of each word put into the new list

  Returns:
    a new list only containing words from the original list with the desired length'''
  matched_lst = []
  for word in word_lst:
    if len(word) == word_len:
      matched_lst.append(word)
  return matched_lst

def hamming_distance(word1, word2):
  '''takes two words and counts the number of different characters between them

  Parameters:
    word1 (str): any word
    word2 (str): another word

  Returns:
    the number of different characters between the two words'''
  if len(word1) != len(word2):
    return None
  count = 0
  for i in range(len(word1)):
    if word1[i] != word2[i]:
      count = count + 1
  return count

def rebuild_path(first, last, reached_from):
  '''takes two keys from a dictionary and maps the quickest path from a start word to an end word

  Parameters:
    first (str): the first word in the desired path
    last (str): the final word in the desired path
    reached_from: a dictionary mapping of a word ladder

  Return:
    a list containing a path from the first word to the last'''
  path = [last]
  current = last
  while first not in path:
    for key in reached_from:
      if current == key:
        path.append(reached_from[key])
        current = reached_from[key]
  path.reverse()
  return path

def get_words(file_name):
  '''takes a file containg a word per line and puts those words into a list

  Parameter:
    file_name (file): the file containing the words

  Returns:
    a list containing the words from the file'''
  words = []
  file = open(file_name)
  lines = file.readlines()

  for line in lines:
    line = line.strip()
    words.append(line)
  file.close()
  return words

def generate_adjacent(word):
  '''takes a word and generates a list of similar words that have a hamming distance of 1

  Parameter:
    word (str): a word whose adjacents will be generated

  Returns:
    a list of words with a hamming distance of 1 from the original word that doesnt include the original'''
  adjacent = []
  for i in range(len(word)):
    #turn string into list since lists are mutable
    char_list = list(word)
    for letter in string.ascii_lowercase:
      char_list[i] = letter
      word1 = ''.join(char_list)
      #not allowing the original word to be included in adjacents
      if word1 != word:
        adjacent.append(word1)
  return adjacent








def test_A2_utils():
  dict = {'act': 'art', 'art': 'act', 'are': 'art', 'arm': 'art', 'age': 'are', 'ate': 'are', 'aim': 'arm', 'ago': 'age', 'him': 'aim', 'aid': 'aim', 'air': 'aim', 'hid': 'him', 'his': 'him', 'hit': 'him', 'bid': 'aid', 'did': 'aid', 'kid': 'aid', 'rid': 'aid', 'add': 'aid', 'and': 'aid', 'had': 'hid', 'has': 'his', 'bit': 'hit', 'fit': 'hit', 'sit': 'hit', 'hat': 'hit', 'hot': 'hit', 'bad': 'bid', 'bed': 'bid', 'big': 'bid', 'die': 'did', 'red': 'rid', 'odd': 'add', 'end': 'and', 'any': 'and', 'mad': 'had', 'sad': 'had', 'gas': 'has', 'was': 'has', 'bet': 'bit', 'but': 'bit', 'fix': 'fit', 'sat': 'sit', 'set': 'sit', 'six': 'sit', 'cat': 'hat', 'eat': 'hat', 'got': 'hot', 'lot': 'hot', 'not': 'hot', 'how': 'hot', 'ban': 'bad', 'bar': 'bad', 'fed': 'bed', 'led': 'bed', 'bug': 'big', 'lie': 'die', 'tie': 'die', 'doe': 'die', 'due': 'die', 'old': 'odd', 'mod': 'mad', 'man': 'mad', 'map': 'mad', 'may': 'mad', 'saw': 'sad', 'say': 'sad', 'gap': 'gas', 'war': 'was', 'way': 'was', 'get': 'bet', 'let': 'bet', 'met': 'bet', 'net': 'bet', 'yet': 'bet', 'cut': 'but', 'out': 'but', 'put': 'but', 'bus': 'but', 'buy': 'but', 'mix': 'fix', 'see': 'set', 'sex': 'set', 'can': 'cat', 'car': 'cat', 'log': 'lot', 'low': 'lot', 'nor': 'not', 'now': 'not', 'row': 'how', 'fan': 'ban', 'ran': 'ban', 'van': 'ban', 'far': 'bar', 'few': 'fed', 'leg': 'led', 'the': 'tie', 'tin': 'tie', 'dog': 'doe', 'men': 'man', 'day': 'may', 'lay': 'may', 'pay': 'may', 'law': 'saw', 'raw': 'saw', 'why': 'way', 'new': 'net', 'yes': 'yet', 'cup': 'cut', 'our': 'out', 'guy': 'buy', 'boy': 'buy', 'she': 'see', 'for': 'nor', 'fun': 'fan', 'run': 'ran', 'win': 'tin', 'ten': 'tin', 'pen': 'men', 'dry': 'day', 'who': 'why', 'gun': 'guy', 'joy': 'boy', 'box': 'boy', 'sun': 'fun', 'won': 'win'}

  words = get_words('words.txt')
  print(x_length_words(words, 4))
  print(hamming_distance('seed', 'head'))
  print(rebuild_path('act', 'won', dict))
  print(generate_adjacent('bat'))


# Graphing Interface

To provide the player with a visualization to see their path from a starting word to a provided target word, a graphmaker class is used to create a binary search tree to keep track of all player inputs.

In [6]:
#Creating a graph interface that highlights nodes, edges, and adjacents
#Used in a seperate group project
#Kevin Schoefernacker and Jake Kougan
#Nov 20, 2023


class GraphMaker:
  def __init__(self):
    '''
    used to create the methods to modify the graph
    Also stores edges and nodes to be highlighted

    Parameters:
      None

    Returns:
      None
    '''
    self._G = nx.Graph()
    self._highedge = dict()
    self._highnode = dict()

  def add_edge(self,node1,node2):
    '''
    Adds an edge between two indicated nodes

    Parameters:
      node1,node2 = nodes in graph, added if not already existent

    Returns:
      None
    '''
    self._G.add_edge(node1, node2)

  def draw_me(self):
    '''
    highlights the nodes and edges in __init__ and plots the graph

    Parameters:
      None

    Returns:
      None
    '''
    pos = nx.spring_layout(self._G)
    options = {
      "node_size" : 1000,
    "node_color" : "white",
    "linewidths" : 0,
    "edgecolors" : "black"
    }
    nx.draw_networkx(self._G,pos,**options)
    for color in self._highedge:
      nx.draw_networkx_edges(self._G,pos,edgelist = self._highedge[color],edge_color = color)
    for color in self._highnode:
      nx.draw_networkx_nodes(self._G,pos,nodelist = self._highnode[color],node_color = color, node_size = 1000)

    plt.tight_layout()
    plt.show()

  def highlight_edge(self, node1, node2, color):
    '''
    Adds an edge to self._highedge to be highlighted

    Parameters:
     node 1, node 2: The nodes which will be connected by a highlighted edge
     color:the color of the highlighted edge

    Returns:
      None
    '''
    #Checks if both nodes are in graph
    if (node1,node2) not in self._G.edges:
      raise KeyError
    if color not in self._highedge:
      self._highedge[color] = [(node1,node2)]
    else:
      self._highedge[color].append((node1,node2))


  def highlight_node(self, node, color):
    '''
    Adds an node to self._highnode to be highlighted

    Parameters:
     node 1: The node which will be highlighted
     color: the color of the highlighted node

    Returns:
      None
    '''
    #Checks if node is in graph
    if node not in self._G.nodes:
      raise KeyError
    if color not in self._highnode:
      self._highnode[color] = [node]
    else:
      self._highnode[color].append(node)


  def highlight_path(self,nodelist, color):
    '''
    Adds edges according to a path to self._highedge to be highlighted

    Parameters:
     nodelist: The path which will be connected by a highlighted edge
     color:the color of the highlighted path

    Returns:
      None
    '''
    edges = []
    for i in range(0,len(nodelist)-1):
      edges.append((nodelist[i],nodelist[i+1]))
    #Checks if all nodes in path are present
    for edge in edges:
      if edge not in self._G.edges:
        raise KeyError
    for node in nodelist:
      if node not in self._G.nodes:
        raise KeyError
    for edge in edges:
      node1 = edge[0]
      node2 = edge[1]
      self.highlight_edge(node1,node2,color)


  def highlight_adjacent(self, node, color):
    '''
    Adds all edges connected to a node to self._highedge to be highlighted

    Parameters:
     node: The nodes from which all all adjacent nodes will be connected by a highlighted edge
     color:the color of the highlighted edge

    Returns:
      None
    '''
    #Checks if node is in graph
    if node not in self._G.nodes:
      raise KeyError
    adjacent = []
    for edge in self._G.edges:
      if node in edge:
        adjacent.append(edge[0])
        adjacent.append(edge[1])
    adjacent = list(set(adjacent))
    adjacent.remove(node)
    for node in adjacent:
      self.highlight_node(node,color)

def test_GraphMaker():
  G = GraphMaker()
  G.add_edge(1,2)
  G.add_edge(2,3)
  G.add_edge(3,4)
  G.add_edge(4,1)
  G.add_edge(2,4)
  G.add_edge(4,4)
  G.add_edge(2,5)
  G.add_edge(3,5)
  G.highlight_edge(1,2,"red")
  G.highlight_edge(3,4,"blue")
  G.highlight_node(1,"blue")
  G.highlight_path([3,2,4],"orange")
  G.highlight_adjacent(5,"pink")
  G.draw_me()

# Creating the game
Below features helper functions that are necessary in the creation of game as well as the game itself. Breadth first search is implemted to provide a dictionary mapping of the start word to the target word and any player submitted guesses in between. A shortest ladder function is used to find the fastest possible word ladder by using BFS and adding each word from the dictionary mapping to a graph that will be compared against the players own path from start word to target word.

In [59]:
#Creating and Visualizing Word Ladders
#Assignment 2
#Author: Jake Kougan
#Date: December 5th, 2023

def bfs(word1, word2, word_lst):
  '''runs breadth first search on a list of words, creating a dictionary mapping starting from the first given word to the last

  Parameters:
    word1 (str): starting word
    word2 (str): ending word
    word_lst (list): a list of words that will be traversed through

  Returns:
    a dictionary mapping of the list traversal from the first word to the last'''
  #Takes the place of the graph
  valid_words = x_length_words(word_lst, len(word1))
  #Takes the place of to_visit
  queue = deque()
  #Takes the place of visited
  queue.append(word1)
  reached = {word1: 'None'}
  while len(queue) != 0:
    current = queue.popleft()
    for word in valid_words:
      if word not in reached and hamming_distance(current, word) == 1:
        queue.append(word)
        reached[word] = current
    if word2 in reached:
      return reached

def shortest_ladder(first, last, word_lst, visualize = True):
  '''creates a visualization of adjacent words while highlighting the quickest path from the start word to the last

  Parameters:
    first (str): the starting word in the mapping
    last (str): final word in the mapping
    word_lst (list): all possible words in traversal
    visualize (Boolean): creates the graph

  Returns:
    the shortest path from the start word to end work as a list and a visualization of all the words in the dictionary with the fastest path highlighted'''
  G = GraphMaker()
  dictionary = bfs_opt(first, last, word_lst)
  #In cases where word length is >= 6 a path is not always found so we must find another
  if dictionary == None:
    while dictionary == None:
        first = random.choice(word_lst)
        last = random.choice(word_lst)
        dictionary = bfs_opt(first, last, word_lst)
  ladder = rebuild_path(first, last, dictionary)
  #Creates the path in a BST
  for node in dictionary:
    G.add_edge(node, dictionary[node])
  G.highlight_path(ladder, 'salmon')
  G.highlight_node(first, 'green')
  G.highlight_node(last, 'red')
  if visualize == True:
    G.draw_me()
  return ladder


def bfs_opt(word1, word2, word_lst):
  '''runs breadth first search with optimazations to make it run faster using sets and generate adjacent

  Parameters:
  word1 (str): starting word
  word2 (str): ending word
  word_lst (list): list of valid words

  Returns:
    a dictionary mapping of the word ladder'''
  #Takes the place of the graph
  valid_words = x_length_words(word_lst, len(word1))
  ladder = {word1: 'None'}
  #switching to a set for faster iteration
  valid_set = set(valid_words)
  #Takes the place of to_visit
  queue = deque()
  queue.append(word1)
  while len(queue) != 0:
    current = queue.popleft()
    adjacents = generate_adjacent(current)
    for word in adjacents:
      if word in valid_set:
        #removing words from the list of available words for faster traversal
        valid_set.discard(word)
        queue.append(word)
        ladder[word] = current
    if word2 in ladder:
      return ladder


def game():
  '''a word ladder game where the player must guess words to create a word ladder with a certain amount of guesses

  Parameters:
    None

  Returns:
    a visualization of the players path along with the quickest path'''
  G = GraphMaker()
  print("Welcome to the Word Ladder Game!")
  length = int(input("Please enter your desired word size. "))
  word_lst = get_words('words.txt')
  valid_lst = x_length_words(word_lst, length)
  start = random.choice(valid_lst)
  end = random.choice(valid_lst)
  path = shortest_ladder(start, end, valid_lst, visualize=False)
  #Just in case we have to reroll the path in case one could not be found
  start = path[0]
  end = path[-1]
  mapping = bfs_opt(start, end, path)

  for m in mapping:
    G.add_edge(m, mapping[m])
  #tracking the path the player takes
  player_path = [start]
  steps = len(path)
  #the program will choose new start and end values if the intial selection was the same
  if start == end:
    start = random.choice(valid_lst)
    end = random.choice(valid_lst)
  print(f"Your goal is to get from {start} to {end} in {steps} steps")
  score = 0
  current = start
  #The inner workings of the game
  while current != end:
    guess = input(f"Current Word: {current}. Target Word: {end}. Enter your next word:  ")
    if hamming_distance(current, guess) > 1:
      print("INVALID WORD! Please enter a valid word")
      guess = player_path[-1]
    score = score + 1
    #creates a bst strand to track the players inputs
    G.add_edge(current,guess)
    player_path.append(guess)
    current = guess
  G.highlight_node(start, 'green')
  G.highlight_node(end, 'red')
  G.highlight_path(path, 'salmon')
  if score != path:
    print(f"Your path: {player_path}. Your Score: {score}")
    print(f" Here was the quickest path: {path}. Score: {steps}")
    return G.draw_me()
  else:
    print(f"Sucess! You reached {end} in {steps} steps. Your Score: {score}")
    return G.draw_me()


def test_A2_ladder():
  words = get_words('words.txt')
  dict = bfs('abash', 'roman', words)
  print(dict)
  print(shortest_ladder('abash', 'roman', dict, visualize=True))
  print(bfs_opt('abash', 'roman', words))
  print(game())

# Playing the Game:
Run the cell below to play the game.

Rules:

1. Upon running the cell, you will be asked to input the legnth of words you want featured in the game.

2. Following step 1, you will be given a target word, start word, and a goal of how many words it will take to reach the target.

3. From there, try to build a path of same sized words from your start word to your target word. The only way to progress is to input words that share a one letter difference from the current word. If this is not followed you will be asked to try again.

4. Once the target word is reached, the game will end showing you a visualization of your path to the word and, if applicable, the quickest path from start word to target word.

Scoring:

All games begin with a score of 0 and a goal score. For each word added to the path, score is increased by 1.



In [61]:
print(game())

Welcome to the Word Ladder Game!
Please enter your desired word size. 6
Your goal is to get from nouses to ageing in 17 steps


KeyboardInterrupt: Interrupted by user